In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')
import pandas as pd
import numpy as np
import datetime
import os
from fastcache import lru_cache
import dcube as dc
%run Quant_useful_packages.ipynb
pro = dc.pro_api('')
all_days = get_all_trade_days()

In [82]:
# 获取outstanding balance数据：尚未偿还的可转债债券的本金余额

df_ccb_amount = pro.query('ccb_amount',fields='code,ann_date,outstanding_balance')
index = ['ann_date','code']
outstanding_balance_df = df_ccb_amount.groupby(index).head(1).set_index(index).unstack()['outstanding_balance']
outstanding_balance_df.index = pd.to_datetime(outstanding_balance_df.index).date
date_s = pd.to_datetime('20100101').date()
date_e = get_near_trade_day(datetime.date.today()-datetime.timedelta(1))
date_list = [date for date in all_days if date_s<=date<=date_e]
outstanding_balance_df = outstanding_balance_df.reindex(index=date_list).fillna(method='ffill')
outstanding_balance_df = outstanding_balance_df[outstanding_balance_df.index>=datetime.date(2016,1,1)]
ccb_code_list = pd.read_csv('C:\\Users\\nibh\\Desktop\\FZ\\CCB\\CCB_stock_code.csv')['ccb_code'].tolist()
outstanding_balance_df = outstanding_balance_df.reindex(columns=ccb_code_list)
outstanding_balance_df.to_csv('C:\\Users\\nibh\\Desktop\\FZ\\CCB\\CCB__outstanding_balance.csv')

code,128015.SZ,117128.SZ,117117.SZ,S72466.IOC,117137.SZ,117164.SZ,128108.SZ,123001.SZ,117127.SZ,117118.SZ,...,123159.SZ,113525.SH,113638.SH,113656.SH,113580.SH,113625.SH,113539.SH,113600.SH,113043.SH,113644.SH
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
@lru_cache()
def get_index_stocks_multdays(index_pool, start_date: str, end_date: str):
    start_date = pd.to_datetime(str(start_date)).date()
    end_date = pd.to_datetime(str(end_date)).date()
    trade_days = [date for date in get_all_trade_days() if start_date<=date<=end_date]
    index_info = get_dcindex_df(index_pool)
    pool_multdays = dict.fromkeys(trade_days)
    for date in trade_days:
        date_str = datetime.date.strftime(date, '%Y%m%d')
        df_one_day = index_info[(index_info['in_date']<=date_str) & (~(index_info['out_date']<=date_str))]
        '''
        (~(a>b))和(a<=b)在处理空值上的区别：a或b是NaN，计算(~(a>b))时a>b返回NaN，取反后仍是NaN
        计算(a<=b)是返回False，因为NaN与任何值的比较结果都是 False
        '''
        pool_multdays[date] = df_one_day['con_code'].tolist()
    return pool_multdays

# 获取正股代码和对应的可转债代码
def get_ccb_stock_code():
    date_s  ='20040101'
    date_e = get_near_trade_day(datetime.date.today()-datetime.timedelta(1))
    winda_df = get_index_stocks_multdays('881001.WI',date_s,date_e)
    all_stocks = []
    for stock_list in winda_df.values():
        all_stocks.extend(stock_list)
    all_stocks_incl_delist = list(set(all_stocks))
    df_a_desc = pro.query('a_desc',fields='code,compcode')
    df_a_desc = df_a_desc[df_a_desc['code'].isin(all_stocks_incl_delist)]
    df_a_desc_tmp = df_a_desc[['code','compcode']].rename(columns={'code':'stock_code'})
    df_ccb_issuance = pro.query('ccb_issuance',fields='code,comp_code')
    df_ccb_issuance_tmp = df_ccb_issuance[pd.notnull(df_ccb_issuance['code'])][['code','comp_code']]
    df_ccb_issuance_tmp = df_ccb_issuance_tmp.rename(columns={'code':'ccb_code'})
    df_com= pd.merge(df_a_desc_tmp,df_ccb_issuance_tmp,left_on='compcode',right_on='comp_code')
    df_com = df_com[['stock_code','ccb_code']]
    return df_com
df_code = get_ccb_stock_code()

In [17]:
# len(set(df_code.index.tolist()))
len(df_code.set_index('ccb_code')['stock_code'].to_dict())

1383

In [8]:
# 从股票因子数据中提取对应的可转债因子数据
ccb_code_list = pd.read_csv('C:\\Users\\nibh\\Desktop\\FZ\\CCB\\CCB_stock_code.csv')['ccb_code'].tolist()
date_s = pd.to_datetime('20100101').date()
date_e = get_near_trade_day(datetime.date.today()-datetime.timedelta(1))

def convert_factors_from_stocks_to_ccb():
    start_date = '20160101'
    end_date = get_near_trade_day(datetime.date.today()-datetime.timedelta(1))
    df_code = get_ccb_stock_code()
    ccb_to_stock_dict = df_code.set_index('ccb_code')['stock_code'].to_dict()   # 字典，键是可转债名，值是正股名
    factor_list = [
            'EODP__S_DQ_CLOSE',
            'EODDI__S_VAL_MV',
            'EODP__S_DQ_PCTCHANGE',
            'EODP__S_DQ_AMOUNT',
       #     'EODP__S__STD240D',
            'EODP__S_DQ_HIGH',
            'EODP__S_DQ_LOW'
    ]
    
    winda_df = get_index_stocks_multdays('881001.WI',date_s,date_e)
    all_stocks = []
    for stock_list in winda_df.values():
        all_stocks.extend(stock_list)
    stock_pool = list(set(all_stocks))
    
    for f in factor_list:
        df = get_single_factor_values(f,datetime.datetime.strptime(start_date, '%Y%m%d').date(),end_date,stock_pool)
        df_ccb = {}
        for ccb_code,stock_code in ccb_to_stock_dict.items():
            if stock_code in df.columns:
                series = df[stock_code]
                df_ccb[ccb_code] = series
        df_ccb = pd.DataFrame(df_ccb)
        df_ccb = df_ccb.reindex(columns=ccb_code_list)
        path = 'C:\\Users\\nibh\\Desktop\\FZ\\CCB'
        file_name = '{}\\CCB__S__{}.csv'.format(path,f)
        print(file_name)
        df_ccb.to_csv(file_name)
    return df_ccb

df_ccb = convert_factors_from_stocks_to_ccb()

C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODP__S_DQ_CLOSE.csv
C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODDI__S_VAL_MV.csv
C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODP__S_DQ_PCTCHANGE.csv
C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODP__S_DQ_AMOUNT.csv
C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODP__S_DQ_HIGH.csv
C:\Users\nibh\Desktop\FZ\CCB\CCB__S__EODP__S_DQ_LOW.csv


In [9]:
path = 'C:\\Users\\nibh\\Desktop\\FZ\\CCB'
list_close = []
list_pctchange = []
list_amount = []
list_high = []
list_low = []
date_s = pd.to_datetime('20160101').date()
date_e = pd.to_datetime('20230906').date()
date_list = [date for date in all_days if date_s<=date<=date_e]
for ccb_code in tqdm(ccb_code_list):
    df_cb_prices = pro.query('cb_eod_prices',code=ccb_code,fields='trade_date,code,pctchange,amount,close,high,low',start_date='20160101')
    if df_cb_prices.empty:
        continue
    else:
        df_cb_prices = df_cb_prices.set_index(['trade_date','code']).unstack()
        df_cb_prices.index = pd.to_datetime(df_cb_prices.index).date
        df_cb_prices = df_cb_prices.reindex(index=date_list)
        list_close.append(df_cb_prices['close'])
        list_pctchange.append(df_cb_prices['pctchange'])
        list_amount.append(df_cb_prices['amount'])
        list_high.append(df_cb_prices['high'])
        list_low.append(df_cb_prices['low'])
df_cb_prices_close = pd.concat(list_close,axis=1).reindex(columns=ccb_code_list)
df_cb_prices_amount = pd.concat(list_amount,axis=1).reindex(columns=ccb_code_list)
df_cb_prices_pctchange = pd.concat(list_pctchange,axis=1).reindex(columns=ccb_code_list)
df_cb_prices_high = pd.concat(list_high,axis=1).reindex(columns=ccb_code_list)
df_cb_prices_low = pd.concat(list_low,axis=1).reindex(columns=ccb_code_list)
df_cb_prices_close.to_csv('{}\\CCB__close.csv'.format(path))
df_cb_prices_amount.to_csv('{}\\CCB__amount.csv'.format(path))
df_cb_prices_pctchange.to_csv('{}\\CCB__pctchange.csv'.format(path))
df_cb_prices_high.to_csv('{}\\CCB__high.csv'.format(path))
df_cb_prices_low.to_csv('{}\\CCB__low.csv'.format(path))

100%|██████████████████████████████████████████████████████████████████████████████| 1384/1384 [07:36<00:00,  3.03it/s]


In [178]:
df_ccb_valuation = pro.query('ccb_valuation',fields='trade_date,code,strbpremium,strbpremium_ratio,strbvalue,conv_price,conv_premium,\
conv_ratio,conv_value,convpremium_ratio,ptm,ytm')
df_ccb_valuation = df_ccb_valuation[df_ccb_valuation['trade_date']>='20160101']
df_ccb_valuation_unstack = df_ccb_valuation.set_index(['trade_date','code']).unstack()
df_ccb_valuation_unstack.index = pd.to_datetime(df_ccb_valuation_unstack.index).date
df_ccb_valuation_unstack = df_ccb_valuation_unstack.reindex(index=date_list)

file_name = '{}\\CCB__strbpremium.csv'.format(path)  # 纯债溢价（转债价格-纯债价值）
df_ccb_valuation_unstack['strbpremium'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__strbpremium_ratio.csv'.format(path)
df_ccb_valuation_unstack['strbpremium_ratio'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__strbvalue.csv'.format(path)  # 纯债价值（∑各期息票现金流折现+本金折现）
df_ccb_valuation_unstack['strbvalue'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__conv_price.csv'.format(path)  # 转股价
df_ccb_valuation_unstack['conv_price'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__conv_premium.csv'.format(path)
df_ccb_valuation_unstack['conv_premium'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__conv_ratio.csv'.format(path)
df_ccb_valuation_unstack['conv_ratio'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__conv_value.csv'.format(path)
df_ccb_valuation_unstack['conv_value'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__convpremium_ratio.csv'.format(path)
df_ccb_valuation_unstack['convpremium_ratio'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__ptm.csv'.format(path)
df_ccb_valuation_unstack['ptm'].reindex(columns=ccb_code_list).to_csv(file_name)

file_name = '{}\\CCB__ytm.csv'.format(path)
df_ccb_valuation_unstack['ytm'].reindex(columns=ccb_code_list).to_csv(file_name)

In [3]:
# 提取陈cache_data_ccb因子
ccb_code_list = pd.read_csv('C:\\Users\\nibh\\Desktop\\FZ\\CCB\\CCB_stock_code.csv')['ccb_code'].tolist()
date_s = pd.to_datetime('20160101').date()
date_e = pd.to_datetime('20230626').date()
date_list = [date for date in all_days if date_s<=date<=date_e]

path = 'C:\\Users\\nibh\\Desktop\\FZ\\CCB\\cache_data_ccb'
path2 = r'C:\Users\nibh\Desktop\FZ\CCB'
i='CCB__convpremium_ratio_add_close'
file_name = '{}\\{}.csv'.format(path,i)
df = pd.read_csv(file_name,index_col=0)
df.index = pd.to_datetime(df.index).date
#df.index = [datetime.datetime.strptime(str(j),'%Y%m%d').date() for j in df.index]
df = df.reindex(index=date_list,columns=ccb_code_list)
df.to_csv('{}\{}.csv'.format(path2,i))
    